# Análise de dados dos jogos do Flamengo 2024

In [39]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd 
import csv

In [3]:
# Importando os arquivos
jogos = pd.read_csv('../data/processed/matches_FLA2024.csv', parse_dates=['kick-off'])
gols = pd.read_csv('../data/processed/goal_scorers_2024.csv')

## Estatísticas 

In [5]:
# Calculando os gols pró, sofridos e saldo de gols
gols_pro = jogos.gols_casa.sum() + jogos.gols_fora.sum()
gols_contra = jogos.gols_sofridos_casa.sum() + jogos.gols_sofridos_fora.sum()
saldo_gols = gols_pro - gols_contra

In [8]:
# Obtendo o número de vitórias, derrotas, empates e total de jogos
vitorias = jogos['resultado'].value_counts()[0]
derrotas = 0 #jogos['resultado'].value_counts()[1]
empates = jogos['resultado'].value_counts()[1]
total_jogos = vitorias + empates + derrotas

# Gols pro e sofridos por jogo
gols_per_jogo = round(gols_pro/total_jogos,2)
gols_sofrido_jogo = round(gols_contra/total_jogos,2)

# Calculando o aproveitamento
aproveitamento = round((3*vitorias + 1*empates + 0*derrotas)/(3*total_jogos)*100,2)
aproveitamento_pontos = (3*vitorias + 1*empates + 0*derrotas)

In [52]:
jogos

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,torneio,locale,resultado,gols_casa,gols_sofridos_casa,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,Casa,Vitória,4,0,0,0,Sim,Não,1
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227,International Friendly Game,Fora,Vitória,0,0,2,0,Sim,Não,2
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428,Carioca,Fora,Empate,0,0,1,1,Sim,Sim,3
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678,International Friendly Game,Fora,Empate,0,0,1,1,Sim,Sim,4
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211,Carioca,Fora,Empate,0,0,0,0,Não,Não,5
5,CA4,Sampaio Corrêa-RJ,Flamengo,0,2,Carioca - Round 4,2024-01-31 21:30:00,Mangueirão-PA,47644,Carioca,Fora,Vitória,0,0,2,0,Sim,Não,6
6,CA5,Vasco,Flamengo,0,0,Carioca - Round 5,2024-02-04 19:00:00,Maracanã-RJ,56318,Carioca,Fora,Empate,0,0,0,0,Não,Não,7
7,CA6,Flamengo,Botafogo,1,0,Carioca - Round 6,2024-02-07 21:30:00,Maracanã-RJ,46798,Carioca,Casa,Vitória,1,0,0,0,Sim,Não,8
8,CA7,Flamengo,Volta Redonda,3,0,Carioca - Round 7,2024-02-10 16:00:00,Maracanã-RJ,42373,Carioca,Casa,Vitória,3,0,0,0,Sim,Não,9
9,CA8,Bangu-RJ,Flamengo,0,3,Carioca - Round 8,2024-02-15 21:30:00,Arena Batistão-SE,15575,Carioca,Fora,Vitória,0,0,3,0,Sim,Não,10


In [53]:
jogos_marcou = jogos.marcou_gol.value_counts()[0]
jogos_não_marcou = jogos.marcou_gol.value_counts()[1]

jogos_sofreu = jogos.sofreu_gol.value_counts(ascending=True)[0]
jogos_não_sofreu = jogos.sofreu_gol.value_counts(ascending=True)[1]

In [54]:
stats = pd.DataFrame(jogos[['id_match', 'marcou_gol', 'sofreu_gol', 'kick-off', 'resultado', 'locale', 'jogos_cumulativos']])

stats['gols_marcados'] = jogos.gols_casa + jogos.gols_fora
stats['gols_sofridos'] = jogos.gols_sofridos_casa + jogos.gols_sofridos_fora

stats['gols_marcados_acumulados'] = stats['gols_marcados'].cumsum()
stats['gols_sofridos_acumulados'] = stats['gols_sofridos'].cumsum()
stats['diferença_gols_acumulados'] = stats['gols_marcados_acumulados'] - stats['gols_sofridos_acumulados']

In [55]:
print(
"""
Jogos: {0}
Vitórias: {1}
Empates: {2}
Derrotas: {3} 
Gols pro: {4}
Gols contra: {5}
Saldo de gols: +{6}
Gols/jogo: {7}
Gols sofridos/jogo: {8}
Jogos em que marcou: {9}
Jogos em que sofreu gol: {10} 
Aproveitamento: {11}%
Aproveitamento pontos: {12}/{13}
""".format(total_jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo_gols,gols_per_jogo,
           gols_sofrido_jogo,jogos_marcou,jogos_sofreu,aproveitamento,aproveitamento_pontos, 3*total_jogos))


Jogos: 11
Vitórias: 7
Empates: 4
Derrotas: 0 
Gols pro: 21
Gols contra: 2
Saldo de gols: +19
Gols/jogo: 1.91
Gols sofridos/jogo: 0.18
Jogos em que marcou: 9
Jogos em que sofreu gol: 2 
Aproveitamento: 75.76%
Aproveitamento pontos: 25/33



### Criando tabelas com o número de gols e assistências

In [30]:
# Fazendo o merge dos dataframes jogos e gols
df = pd.merge(jogos, gols, on = 'id_match')
df.head()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,torneio,...,resultado,gols_casa,gols_sofridos_casa,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos,gols,assist
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,Vitória,4,0,0,0,Sim,Não,1,Léo Pereira,Arrascaeta
1,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,Vitória,4,0,0,0,Sim,Não,1,Pedro,Gerson
2,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,Vitória,4,0,0,0,Sim,Não,1,Everton Cebolinha,NaN
3,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,Vitória,4,0,0,0,Sim,Não,1,Varela,Luiz Araújo
4,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227,International Friendly Game,...,Vitória,0,0,2,0,Sim,Não,2,Pedro,NaN


In [31]:
# Quantidade de gols por jogador
df_gols_counts = pd.DataFrame(gols.gols.value_counts())
df_gols_counts = df_gols_counts.reset_index()
df_gols_counts.columns = ['Jogador', 'Gols'] 

# Quantidade de assistência por jogador

df_assist_counts = pd.DataFrame(gols.assist.value_counts())
df_assist_counts = df_assist_counts.reset_index()
df_assist_counts.columns = ['Jogador', 'Assistências'] 

new_df = pd.concat([df_assist_counts, df_gols_counts], axis=1)
print(new_df.to_markdown(index=False))

| Jogador           |   Assistências | Jogador           |   Gols |
|:------------------|---------------:|:------------------|-------:|
| Arrascaeta        |              3 | Pedro             |      8 |
| Luiz Araújo       |              3 | Léo Pereira       |      2 |
| Gerson            |              2 | Everton Cebolinha |      2 |
| Lorran            |              1 | Gabi              |      2 |
| Varela            |              1 | Arrascaeta        |      2 |
| Ayrton Lucas      |              1 | Varela            |      1 |
| Igor Jesus        |              1 | Thiaguinho        |      1 |
| Everton Cebolinha |              1 | Bruno Henrique    |      1 |
| de la Cruz        |              1 | Ayrton Lucas      |      1 |
| nan               |            nan | Luiz Araujo       |      1 |


In [32]:
# Unificando as tabelas gols e assistências
uni_df = pd.merge(df_gols_counts, df_assist_counts, on = 'Jogador', how ="left").fillna(0).head(15)

print(uni_df.to_markdown(index=False))

| Jogador           |   Gols |   Assistências |
|:------------------|-------:|---------------:|
| Pedro             |      8 |              0 |
| Léo Pereira       |      2 |              0 |
| Everton Cebolinha |      2 |              1 |
| Gabi              |      2 |              0 |
| Arrascaeta        |      2 |              3 |
| Varela            |      1 |              1 |
| Thiaguinho        |      1 |              0 |
| Bruno Henrique    |      1 |              0 |
| Ayrton Lucas      |      1 |              1 |
| Luiz Araujo       |      1 |              0 |


In [35]:
stats.to_csv('../data/processed/stats_FLA2024.csv', index=False)